In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 127/30 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = 3 * TR_C * TR_R
TR_R = tau / (3*TR_C)




# Logic Gates
g_inv, p_inv = 1, 323/127

g_nand2, g_nand3, g_nand6 = 4/3 , 5/3, 8/3
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 5/3, 7/3
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [ ]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

# NMOS width of repeater derived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/(3*metal_r*TR_C) ) ** (1/2)

C_out = (WL_C + cell_C)/N + 3 * W * TR_C
C_in = 5 * 1e-15 * 32

H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = [1,2,4,8,16,32,64,128], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,1,2,4,8,16,32,64,128
NAND6-INV,3.684282,3.195501,2.726319,2.306970,1.995778,1.857608,1.898157,2.056670
NAND3-NOR2,3.713729,3.224948,2.755766,2.336416,2.025225,1.887055,1.927604,2.086117


In [ ]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.842075e-10,1.529296e-10,1.313961e-10,1.172633e-10,1.091344e-10,1.060504e-10,1.069251e-10,1.105901e-10
NAND3-NOR2 Stage2,1.649186e-10,1.329957e-10,1.110182e-10,9.659394e-11,8.829736e-11,8.514979e-11,8.604248e-11,8.978311e-11


In [ ]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,1,2,4,8,16,32,64,128
NAND6-INV Stage4,1.576131e-10,1.481523e-10,1.404607e-10,1.345683e-10,1.307172e-10,1.291359e-10,1.295922e-10,1.314384e-10
NAND3-NOR2 Stage4,1.367025e-10,1.271447e-10,1.193742e-10,1.134214e-10,1.095307e-10,1.079333e-10,1.083942e-10,1.102593e-10


In [ ]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,1,2,4,8,16,32,64,128
NAND6-INV Stage6,1.783409e-10,1.719434e-10,1.664461e-10,1.620124e-10,1.589890e-10,1.577147e-10,1.580845e-10,1.595636e-10
NAND3-NOR2 Stage6,1.568076e-10,1.504101e-10,1.449127e-10,1.404790e-10,1.374557e-10,1.361814e-10,1.365512e-10,1.380303e-10


In [ ]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND6-INV'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,1,2,4,8,16,32,64,128
NAND6-INV Stage8,2.040939e-10,1.988860e-10,1.942853e-10,1.904780e-10,1.878263e-10,1.866940e-10,1.870235e-10,1.883340e-10
NAND3-NOR2 Stage8,1.828886e-10,1.776541e-10,1.730298e-10,1.692031e-10,1.665378e-10,1.653998e-10,1.657309e-10,1.670481e-10


In [ ]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.842075e-10,1.529296e-10,1.313961e-10,1.172633e-10,1.091344e-10,1.060504e-10,1.069251e-10,1.105901e-10
NAND3-NOR2 Stage2,1.649186e-10,1.329957e-10,1.110182e-10,9.659394e-11,8.829736e-11,8.514979e-11,8.604248e-11,8.978311e-11
NAND6-INV Stage4,1.576131e-10,1.481523e-10,1.404607e-10,1.345683e-10,1.307172e-10,1.291359e-10,1.295922e-10,1.314384e-10
NAND3-NOR2 Stage4,1.367025e-10,1.271447e-10,1.193742e-10,1.134214e-10,1.095307e-10,1.079333e-10,1.083942e-10,1.102593e-10
NAND6-INV Stage6,1.783409e-10,1.719434e-10,1.664461e-10,1.620124e-10,1.589890e-10,1.577147e-10,1.580845e-10,1.595636e-10
NAND3-NOR2 Stage6,1.568076e-10,1.504101e-10,1.449127e-10,1.404790e-10,1.374557e-10,1.361814e-10,1.365512e-10,1.380303e-10
NAND6-INV Stage8,2.040939e-10,1.988860e-10,1.942853e-10,1.904780e-10,1.878263e-10,1.866940e-10,1.870235e-10,1.883340e-10
NAND3-NOR2 Stage8,1.828886e-10,1.776541e-10,1.730298e-10,1.692031e-10,1.665378e-10,1.653998e-10,1.657309e-10,1.670481e-10


In [ ]:
segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (3*TR_C*W) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 15 *1e-12 #measured
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.21895368e-09, 2.80395823e-09, 1.64761154e-09, 1.18563729e-09,
       1.18407583e-09, 1.62194366e-09, 2.67829071e-09, 4.81688653e-09])

In [ ]:
delay_total = delay_decoder.copy()
delay_total = delay_total + delay_WL
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.403161e-09,2.956888e-09,1.779008e-09,1.302901e-09,1.293210e-09,1.727994e-09,2.785216e-09,4.927477e-09
NAND3-NOR2 Stage2,5.383872e-09,2.936954e-09,1.758630e-09,1.282231e-09,1.272373e-09,1.707093e-09,2.764333e-09,4.906670e-09
NAND6-INV Stage4,5.376567e-09,2.952111e-09,1.788072e-09,1.320206e-09,1.314793e-09,1.751080e-09,2.807883e-09,4.948325e-09
NAND3-NOR2 Stage4,5.355656e-09,2.931103e-09,1.766986e-09,1.299059e-09,1.293607e-09,1.729877e-09,2.786685e-09,4.927146e-09
NAND6-INV Stage6,5.397295e-09,2.975902e-09,1.814058e-09,1.347650e-09,1.343065e-09,1.779658e-09,2.836375e-09,4.976450e-09
NAND3-NOR2 Stage6,5.375761e-09,2.954368e-09,1.792524e-09,1.326116e-09,1.321532e-09,1.758125e-09,2.814842e-09,4.954917e-09
NAND6-INV Stage8,5.423048e-09,3.002844e-09,1.841897e-09,1.376115e-09,1.371902e-09,1.808638e-09,2.865314e-09,5.005221e-09
NAND3-NOR2 Stage8,5.401842e-09,2.981612e-09,1.820641e-09,1.354840e-09,1.350614e-09,1.787343e-09,2.844022e-09,4.983935e-09


In [ ]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.389650e-09,2.961473e-09,1.795227e-09,1.326140e-09,1.320137e-09,1.756226e-09,2.813084e-09,4.953767e-09
std,2.090137e-11,2.392859e-11,2.837767e-11,3.166922e-11,3.354091e-11,3.422566e-11,3.403350e-11,3.321154e-11
min,5.355656e-09,2.931103e-09,1.758630e-09,1.282231e-09,1.272373e-09,1.707093e-09,2.764333e-09,4.906670e-09
25%,5.376365e-09,2.948321e-09,1.776002e-09,1.301940e-09,1.293507e-09,1.729406e-09,2.786318e-09,4.927394e-09
50%,5.390583e-09,2.955628e-09,1.790298e-09,1.323161e-09,1.318162e-09,1.754602e-09,2.811362e-09,4.951621e-09
75%,5.402172e-09,2.977329e-09,1.815704e-09,1.349447e-09,1.344952e-09,1.781580e-09,2.838287e-09,4.978321e-09
max,5.423048e-09,3.002844e-09,1.841897e-09,1.376115e-09,1.371902e-09,1.808638e-09,2.865314e-09,5.005221e-09


In [ ]:
H

array([1.93660148, 0.98347803, 0.51320295, 0.28695607, 0.18640591,
       0.15391213, 0.16281181, 0.20282426])

In [ ]:
f_2

,1,2,4,8,16,32,64,128
NAND6-INV,12.855219,9.160975,6.617652,4.948426,3.988313,3.624063,3.727368,4.160249
NAND3-NOR2,13.120303,9.349881,6.754113,5.050466,4.070554,3.698794,3.804229,4.246036


In [ ]:
W

array([0.64930219, 0.91825196, 1.29860437, 1.83650392, 2.59720874,
       3.67300783, 5.19441749, 7.34601566])

In [ ]:
2*2.59720874

5.19441748

In [ ]:
C_out

array([3.09856236e-13, 1.57356485e-13, 8.21124724e-14, 4.59129706e-14,
       2.98249449e-14, 2.46259411e-14, 2.60498897e-14, 3.24518822e-14])